# AXON: Examples

In [1]:
from axon.api import loads, dumps
from axon.objects import mapping, element, sequence, instance, empty
from axon.objects import Element, Instance, Mapping, Sequence, Empty
from axon.objects import GenericBuilder, register_builder
from axon import dump_as_str, as_unicode, factory, reduce

from datetime import date, time, datetime
from pprint import pprint
from axon.errors import LoaderError

try:
    from cdecimal import Decimal  
except:
    from decimal import Decimal

## Simple values

### Integer values

In [2]:
vs = loads('0 1 -1 32768 -9223372036854775807')
pprint(vs)
print(dumps(vs))

[0, 1, -1, 32768, -9223372036854775807]
0
1
-1
32768
-9223372036854775807


In [3]:
vs = loads('340282366920938463463374607431768211456 -340282366920938463463374607431768211456') # long int value =/-2**128
pprint(vs)
print(dumps(vs))

[340282366920938463463374607431768211456,
 -340282366920938463463374607431768211456]
340282366920938463463374607431768211456
-340282366920938463463374607431768211456


### Float values

In [4]:
vs = loads('0. 0.0 1.0 -1.0 -0.0 3.141528 1.41e2 -1.41e-2')
pprint(vs)
print(dumps(vs))

[0.0, 0.0, 1.0, -1.0, -0.0, 3.141528, 141.0, -0.0141]
0.0
0.0
1.0
-1.0
-0.0
3.141528
141.0
-0.0141


In [5]:
try:
    vs = loads('.12')
except ValueError:
    print('invalid float number')

invalid float number


In [6]:
vs = loads('Infinity -Infinity NaN ∞ -∞ ?')
pprint(vs)
print(dumps(vs))

[inf, -inf, nan, inf, -inf, nan]
∞
-∞
?
∞
-∞
?


In [7]:
try:
    vs = loads('.')
except ValueError:
    print('invalid float number')

invalid float number


### Decimal values

In [8]:
vs = loads('0$ 1$ -1$ 0.$ 0.0$ 1.0$ -1.0$ -0.0$ 3.141528$ 1.41e2$ -1.41e-2$')
pprint(vs)
print(dumps(vs))

[Decimal('0'),
 Decimal('1'),
 Decimal('-1'),
 Decimal('0'),
 Decimal('0.0'),
 Decimal('1.0'),
 Decimal('-1.0'),
 Decimal('-0.0'),
 Decimal('3.141528'),
 Decimal('141'),
 Decimal('-0.0141')]
0D
1D
-1D
0D
0.0D
1.0D
-1.0D
-0.0D
3.141528D
141D
-0.0141D


In [9]:
vs = loads('Infinity$ -Infinity$ NaN$ ∞$ -∞$ ?$')
pprint(vs)
print(dumps(vs))

[Decimal('Infinity'),
 Decimal('-Infinity'),
 Decimal('NaN'),
 Decimal('Infinity'),
 Decimal('-Infinity'),
 Decimal('NaN')]
∞D
-∞D
?D
∞D
-∞D
?D


### Unicode text

In [10]:
vs = loads('''"any unicode text
with line breaks
and control symbols \t\n"''')
pprint(vs)
print(dumps(vs))

['any unicode text\nwith line breaks\nand control symbols \t\n']
"any unicode text
with line breaks
and control symbols 	
"


In [11]:
vs = loads('''"text line \
splited \
into chunks"''')
pprint(vs)
print(dumps(vs))

['text line splited into chunks']
"text line splited into chunks"


### Date/Time values

In [12]:
vs = loads('''2012-12-31 12:30 12:30:59 12:30:59.150000 12:30:59+03 2012-12-31T12:30:59.015000-04''')
pprint(vs)
print(dumps(vs, pretty=1))

[datetime.date(2012, 12, 31),
 datetime.time(12, 30),
 datetime.time(12, 30, 59),
 datetime.time(12, 30, 59, 150000),
 datetime.time(12, 30, 59, tzinfo=datetime.timezone(datetime.timedelta(0, 10800))),
 datetime.datetime(2012, 12, 31, 12, 30, 59, 15000, tzinfo=datetime.timezone(datetime.timedelta(-1, 72000)))]
2012-12-31
12:30
12:30:59
12:30:59.150000
12:30:59+03
2012-12-31T12:30:59.015000-04


## Anonymous complex objects

### Simply lines of data

In [13]:
text = u'''
# No Sym Flag Date Time
[1 "a" true 2007-12-20 9:00]
[2 "b" false 2007-12-20 15:00]
[3 "c" false 2007-12-20 00:00]
'''
vs = loads(text)
pprint(vs)

[[1, 'a', True, datetime.date(2007, 12, 20), datetime.time(9, 0)],
 [2, 'b', False, datetime.date(2007, 12, 20), datetime.time(15, 0)],
 [3, 'c', False, datetime.date(2007, 12, 20), datetime.time(0, 0)]]


In [14]:
print(dumps(vs))

[1 "a" true 2007-12-20 09:00]
[2 "b" false 2007-12-20 15:00]
[3 "c" false 2007-12-20 00:00]


In [15]:
print(dumps(vs, pretty=1, hsize=3))

[ 1 "a" true
  2007-12-20 09:00]
[ 2 "b" false
  2007-12-20 15:00]
[ 3 "c" false
  2007-12-20 00:00]


### Simply dicts

In [16]:
text = u'''
{"a":1 "b":2.0 "c":3000$}
{"a":100 "b":200.5 "c":30$ "d":null}
'''
vs = loads(text)
pprint(vs)

[{'a': 1, 'b': 2.0, 'c': Decimal('3000')},
 {'a': 100, 'b': 200.5, 'c': Decimal('30'), 'd': None}]


In [17]:
print(dumps(vs))

{a:1 b:2.0 c:3000D}
{a:100 b:200.5 c:30D d:null}


In [18]:
print(dumps(vs, pretty=1))

{ a: 1
  b: 2.0
  c: 3000D}
{ a: 100
  b: 200.5
  c: 30D
  d: null}


### Simply tuples

In [19]:
text = u'''
(2013-01-01 12:30) 
(2013-01-02 13:00)
(2013-01-03 14:30)
'''
values = loads(text)
pprint(values)

[(datetime.date(2013, 1, 1), datetime.time(12, 30)),
 (datetime.date(2013, 1, 2), datetime.time(13, 0)),
 (datetime.date(2013, 1, 3), datetime.time(14, 30))]


In [20]:
print(dumps(values))

(2013-01-01 12:30)
(2013-01-02 13:00)
(2013-01-03 14:30)


In [21]:
print(dumps(values, pretty=1, hsize=2))

(2013-01-01 12:30)
(2013-01-02 13:00)
(2013-01-03 14:30)


### Simply dicts of lists

In [22]:
text = u'''
{ a: [0 1 2] 
  b: [0. 1.0 2.0] 
  c: [0.$ 1.0$] 
  d: [true] }
{ a: [100 200 300] 
  b: [100. 200. 300.] 
  c: [10$ 20$] 
  s: [false] }
'''
values = loads(text)
pprint(values)

[{'a': [0, 1, 2],
  'b': [0.0, 1.0, 2.0],
  'c': [Decimal('0'), Decimal('1.0')],
  'd': [True]},
 {'a': [100, 200, 300],
  'b': [100.0, 200.0, 300.0],
  'c': [Decimal('10'), Decimal('20')],
  's': [False]}]


In [23]:
print(dumps(values))

{a:[0 1 2] b:[0.0 1.0 2.0] c:[0D 1.0D] d:[true]}
{a:[100 200 300] b:[100.0 200.0 300.0] c:[10D 20D] s:[false]}


In [24]:
print(dumps(values, pretty=1))

{ a: [0 1 2]
  b: [0.0 1.0 2.0]
  c: [0D 1.0D]
  d: [true]}
{ a: [100 200 300]
  b: [100.0 200.0 300.0]
  c: [10D 20D]
  s: [false]}


In [25]:
print(dumps(values, pretty=1, hsize=3))

{ a: [0 1 2]
  b: [0.0 1.0 2.0]
  c: [0D 1.0D]
  d: [true]}
{ a: [100 200 300]
  b: [100.0 200.0 300.0]
  c: [10D 20D]
  s: [false]}


## Named complex objects

### Simply named dicts of tuples

In [26]:
text = u'''
rectangle {
    left_top: (-1.0 1.0)
    left_bottom: (-1.0 -1.0)
    right_top: (1.0 1.0)
    right_bottom: (1.0 -1.0)
}
rectangle {
    left_top: (-2.0 2.0)
    left_bottom: (-2.0 -1.0)
    right_top: (1.0 2.0)
    right_bottom: (1.0 -1.0)
}
'''
values = loads(text)
pprint(values)

[mapping('rectangle', {'right_top': (1.0, 1.0), 'left_bottom': (-1.0, -1.0), 'left_top': (-1.0, 1.0), 'right_bottom': (1.0, -1.0)}),
 mapping('rectangle', {'right_top': (1.0, 2.0), 'left_bottom': (-2.0, -1.0), 'left_top': (-2.0, 2.0), 'right_bottom': (1.0, -1.0)})]


In [27]:
print(values[0])
print(values[1])
print(values[0].name, values[0].mapping)
print(values[0].a.left_bottom, values[0].a.right_top)

mapping('rectangle', {'right_top': (1.0, 1.0), 'left_bottom': (-1.0, -1.0), 'left_top': (-1.0, 1.0), 'right_bottom': (1.0, -1.0)})
mapping('rectangle', {'right_top': (1.0, 2.0), 'left_bottom': (-2.0, -1.0), 'left_top': (-2.0, 2.0), 'right_bottom': (1.0, -1.0)})
rectangle {'right_top': (1.0, 1.0), 'left_bottom': (-1.0, -1.0), 'left_top': (-1.0, 1.0), 'right_bottom': (1.0, -1.0)}
(-1.0, -1.0) (1.0, 1.0)


In [28]:
print(dumps(values))

rectangle{left_bottom:(-1.0 -1.0) left_top:(-1.0 1.0) right_bottom:(1.0 -1.0) right_top:(1.0 1.0)}
rectangle{left_bottom:(-2.0 -1.0) left_top:(-2.0 2.0) right_bottom:(1.0 -1.0) right_top:(1.0 2.0)}


In [29]:
print(dumps(values, pretty=1, hsize=2))

rectangle:
  left_bottom: (-1.0 -1.0)
  left_top: (-1.0 1.0)
  right_bottom: (1.0 -1.0)
  right_top: (1.0 1.0)
rectangle:
  left_bottom: (-2.0 -1.0)
  left_top: (-2.0 2.0)
  right_bottom: (1.0 -1.0)
  right_top: (1.0 2.0)


In [30]:
text2 = dumps(values, pretty=2, hsize=2)
print(text2)

rectangle:
  left_bottom: (-1.0 -1.0)
  left_top: (-1.0 1.0)
  right_bottom: (1.0 -1.0)
  right_top: (1.0 1.0)
rectangle:
  left_bottom: (-2.0 -1.0)
  left_top: (-2.0 2.0)
  right_bottom: (1.0 -1.0)
  right_top: (1.0 2.0)


### Simply named lists of tuples

In [31]:
text = u'''
rectangle {
    (-1.0 1.0) 
    (-1.0 -1.0)
    (1.0 -1.0)
    (1.0 1.0)
}
rectangle {
    (-2.0 2.0)
    (-2.0 -1.0)
    (1.0 -1.0)
    (1.0 2.0)
}
'''
values = loads(text)
pprint(values)

[sequence('rectangle', [(-1.0, 1.0), (-1.0, -1.0), (1.0, -1.0), (1.0, 1.0)]),
 sequence('rectangle', [(-2.0, 2.0), (-2.0, -1.0), (1.0, -1.0), (1.0, 2.0)])]


In [32]:
print(values[0])
print(values[1])
print(values[0].name, values[0].sequence)

sequence('rectangle', [(-1.0, 1.0), (-1.0, -1.0), (1.0, -1.0), (1.0, 1.0)])
sequence('rectangle', [(-2.0, 2.0), (-2.0, -1.0), (1.0, -1.0), (1.0, 2.0)])
rectangle [(-1.0, 1.0), (-1.0, -1.0), (1.0, -1.0), (1.0, 1.0)]


In [33]:
print(dumps(values))

rectangle{(-1.0 1.0) (-1.0 -1.0) (1.0 -1.0) (1.0 1.0)}
rectangle{(-2.0 2.0) (-2.0 -1.0) (1.0 -1.0) (1.0 2.0)}


In [34]:
print(dumps(values, pretty=1))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


In [35]:
print(dumps(values, pretty=1, hsize=2))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


In [36]:
print(dumps(values, pretty=2, hsize=2))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


### Simple file system: attributed tree

In [37]:
text = u'''
folder {
  name: "/"
  folder {
    name: "Library"
    file {name: "book.pdf"}
    file {name: "README"}
    folder {
      name: "Temp"
    }
  }
  folder {
    name: "Desktop"
    file {name: "Notes.txt"}
    file {name: "Recipes.txt"}
  }
}
'''
values = loads(text)
pprint(values)

[element('folder', {'name': '/'}, [element('folder', {'name': 'Library'}, [mapping('file', {'name': 'book.pdf'}), mapping('file', {'name': 'README'}), mapping('folder', {'name': 'Temp'})]), element('folder', {'name': 'Desktop'}, [mapping('file', {'name': 'Notes.txt'}), mapping('file', {'name': 'Recipes.txt'})])])]


In [38]:
val = values[0]
print(val.name)
print(val[0].a.name, val[1].a.name)
print(val[0][0].a.name, val[1][0].a.name)
print(val[1].name, val[1].mapping, val[1].sequence)

folder
Library Desktop
book.pdf Notes.txt
folder {'name': 'Desktop'} [mapping('file', {'name': 'Notes.txt'}), mapping('file', {'name': 'Recipes.txt'})]


In [39]:
print(dumps(values))

folder{name:"/" folder{name:"Library" file{name:"book.pdf"} file{name:"README"} folder{name:"Temp"}} folder{name:"Desktop" file{name:"Notes.txt"} file{name:"Recipes.txt"}}}


In [40]:
text2 = dumps(values, pretty=1)
print(text2)

folder:
  name: "/"
  folder:
    name: "Library"
    file:
      name: "book.pdf"
    file:
      name: "README"
    folder:
      name: "Temp"
  folder:
    name: "Desktop"
    file:
      name: "Notes.txt"
    file:
      name: "Recipes.txt"


In [41]:
values2 = loads(text2)
print(dumps(values, pretty=2))

folder:
  name: "/"
  folder:
    name: "Library"
    file:
      name: "book.pdf"
    file:
      name: "README"
    folder:
      name: "Temp"
  folder:
    name: "Desktop"
    file:
      name: "Notes.txt"
    file:
      name: "Recipes.txt"


### Simple sequence of instances

In [42]:
text = u'''
datarow:
  2000-12-01 00:00 T:12.5 R:3.0
datarow:
  2000-12-01 06:00 T:13.0
datarow:
  2000-12-01 12:00 T:14.0 R:1.0
datarow:
  2000-12-01 18:00 T:12.0
datarow:
  2000-12-02 0:00 T:8.0
'''
values = loads(text)
pprint(values)

[instance('datarow', (datetime.date(2000, 12, 1), datetime.time(0, 0)), {'R': 3.0, 'T': 12.5}),
 instance('datarow', (datetime.date(2000, 12, 1), datetime.time(6, 0)), {'T': 13.0}),
 instance('datarow', (datetime.date(2000, 12, 1), datetime.time(12, 0)), {'R': 1.0, 'T': 14.0}),
 instance('datarow', (datetime.date(2000, 12, 1), datetime.time(18, 0)), {'T': 12.0}),
 instance('datarow', (datetime.date(2000, 12, 2), datetime.time(0, 0)), {'T': 8.0})]


In [43]:
print(dumps(values))

datarow{2000-12-01 00:00 R:3.0 T:12.5}
datarow{2000-12-01 06:00 T:13.0}
datarow{2000-12-01 12:00 R:1.0 T:14.0}
datarow{2000-12-01 18:00 T:12.0}
datarow{2000-12-02 00:00 T:8.0}


In [44]:
print(dumps(values, pretty=1))

datarow:
  2000-12-01 00:00
  R: 3.0
  T: 12.5
datarow:
  2000-12-01 06:00
  T: 13.0
datarow:
  2000-12-01 12:00
  R: 1.0
  T: 14.0
datarow:
  2000-12-01 18:00
  T: 12.0
datarow:
  2000-12-02 00:00
  T: 8.0


In [45]:
print(dumps(values, pretty=1, hsize=2))

datarow:
  2000-12-01 00:00
  R: 3.0
  T: 12.5
datarow:
  2000-12-01 06:00
  T: 13.0
datarow:
  2000-12-01 12:00
  R: 1.0
  T: 14.0
datarow:
  2000-12-01 18:00
  T: 12.0
datarow:
  2000-12-02 00:00
  T: 8.0


In [46]:
print(dumps(values, pretty=2, hsize=2))

datarow:
  2000-12-01 00:00
  R: 3.0
  T: 12.5
datarow:
  2000-12-01 06:00
  T: 13.0
datarow:
  2000-12-01 12:00
  R: 1.0
  T: 14.0
datarow:
  2000-12-01 18:00
  T: 12.0
datarow:
  2000-12-02 00:00
  T: 8.0


### Network of connected units

In [47]:
text = u'''
network {
    nodes {
        &x1 node {label:"x1"}
        &x2 node {label:"x2"}
        &x3 node {label:"x3"}
        &u1 node {label:"u1"}
        &u2 node {label:"u2"}
        &y node {label:"y"}
    }
    units {
        unit {out:(*u1) inp:(*x1 *x2 *x3) weights:(1.0 0.5 0.5)}
        unit {out:(*u2) inp:(*x1 *x2) weights:(1.0 1.5)}
        unit {out:(*y) inp:(*u1 *u2) weights:(0.5 -1.0)}
    }
}
'''
values = loads(text)
pprint(values)

[sequence('network', [sequence('nodes', [mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'}), mapping('node', {'label': 'x3'}), mapping('node', {'label': 'u1'}), mapping('node', {'label': 'u2'}), mapping('node', {'label': 'y'})]), sequence('units', [mapping('unit', {'inp': (mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'}), mapping('node', {'label': 'x3'})), 'weights': (1.0, 0.5, 0.5), 'out': (mapping('node', {'label': 'u1'}),)}), mapping('unit', {'inp': (mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'})), 'weights': (1.0, 1.5), 'out': (mapping('node', {'label': 'u2'}),)}), mapping('unit', {'inp': (mapping('node', {'label': 'u1'}), mapping('node', {'label': 'u2'})), 'weights': (0.5, -1.0), 'out': (mapping('node', {'label': 'y'}),)})])])]


In [48]:
val = values[0]
print('Same objects here:')
print(val[0][0] is val[1][0].a.inp[0])
print(val[0][-1] is val[1][-1].a.out[0])

Same objects here:
True
True


In [49]:
print(dumps(values))

network{nodes{node{label:"x1"} node{label:"x2"} node{label:"x3"} node{label:"u1"} node{label:"u2"} node{label:"y"}} units{unit{inp:(node{label:"x1"} node{label:"x2"} node{label:"x3"}) out:(node{label:"u1"}) weights:(1.0 0.5 0.5)} unit{inp:(node{label:"x1"} node{label:"x2"}) out:(node{label:"u2"}) weights:(1.0 1.5)} unit{inp:(node{label:"u1"} node{label:"u2"}) out:(node{label:"y"}) weights:(0.5 -1.0)}}}


In [50]:
print(dumps(values, crossref=1))

network{nodes{&4 node{label:"x1"} &5 node{label:"x2"} &1 node{label:"x3"} &2 node{label:"u1"} &6 node{label:"u2"} &3 node{label:"y"}} units{unit{inp:(*4 *5 *1) out:(*2) weights:(1.0 0.5 0.5)} unit{inp:(*4 *5) out:(*6) weights:(1.0 1.5)} unit{inp:(*2 *6) out:(*3) weights:(0.5 -1.0)}}}


In [51]:
print(dumps(values, crossref=1, pretty=1, hsize=3))

network:
  nodes:
    &4 node:
      label: "x1"
    &5 node:
      label: "x2"
    &1 node:
      label: "x3"
    &2 node:
      label: "u1"
    &6 node:
      label: "u2"
    &3 node:
      label: "y"
  units:
    unit:
      inp: (*4 *5 *1)
      out: (*2)
      weights: (1.0 0.5 0.5)
    unit:
      inp: (*4 *5)
      out: (*6)
      weights: (1.0 1.5)
    unit:
      inp: (*2 *6)
      out: (*3)
      weights: (0.5 -1.0)


In [52]:
print(dumps(values, crossref=1, pretty=2, hsize=3))

network:
  nodes:
    &4 node:
      label: "x1"
    &5 node:
      label: "x2"
    &1 node:
      label: "x3"
    &2 node:
      label: "u1"
    &6 node:
      label: "u2"
    &3 node:
      label: "y"
  units:
    unit:
      inp: (*4 *5 *1)
      out: (*2)
      weights: (1.0 0.5 0.5)
    unit:
      inp: (*4 *5)
      out: (*6)
      weights: (1.0 1.5)
    unit:
      inp: (*2 *6)
      out: (*3)
      weights: (0.5 -1.0)


### XML-style representation with collections of elements with same name

In [53]:
text = u'''
network {
    nodes {
        &x1 node {label:"x1"}
        &x2 node {label:"x2"}
        &x3 node {label:"x3"}
        &u1 node {label:"u1"}
        &u2 node {label:"u2"}
        &y  node {label:"y"}
    }
    units {
        unit {out:(*u1) inp:(*x1 *x2 *x3) weights:(1.0 0.5 0.5)}
        unit {out:(*u2) inp:(*x1 *x2) weights:(1.0 1.5)}
        unit {out:(*y) inp:(*u1 *u2) weights:(0.5 -1.0)}
    }
}
'''
values = loads(text)
pprint(values)

[sequence('network', [sequence('nodes', [mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'}), mapping('node', {'label': 'x3'}), mapping('node', {'label': 'u1'}), mapping('node', {'label': 'u2'}), mapping('node', {'label': 'y'})]), sequence('units', [mapping('unit', {'inp': (mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'}), mapping('node', {'label': 'x3'})), 'weights': (1.0, 0.5, 0.5), 'out': (mapping('node', {'label': 'u1'}),)}), mapping('unit', {'inp': (mapping('node', {'label': 'x1'}), mapping('node', {'label': 'x2'})), 'weights': (1.0, 1.5), 'out': (mapping('node', {'label': 'u2'}),)}), mapping('unit', {'inp': (mapping('node', {'label': 'u1'}), mapping('node', {'label': 'u2'})), 'weights': (0.5, -1.0), 'out': (mapping('node', {'label': 'y'}),)})])])]


In [54]:
val = values[0]
print(val[0][0] is val[1][0].a['inp'][0])
print(val[0][-1] is val[1][-1].a['out'][0])

True
True


In [55]:
print(dumps(values, crossref=1))

network{nodes{&1 node{label:"x1"} &6 node{label:"x2"} &3 node{label:"x3"} &2 node{label:"u1"} &5 node{label:"u2"} &4 node{label:"y"}} units{unit{inp:(*1 *6 *3) out:(*2) weights:(1.0 0.5 0.5)} unit{inp:(*1 *6) out:(*5) weights:(1.0 1.5)} unit{inp:(*2 *5) out:(*4) weights:(0.5 -1.0)}}}


In [56]:
print(dumps(values, crossref=1, pretty=1, hsize=3))

network:
  nodes:
    &1 node:
      label: "x1"
    &6 node:
      label: "x2"
    &3 node:
      label: "x3"
    &2 node:
      label: "u1"
    &5 node:
      label: "u2"
    &4 node:
      label: "y"
  units:
    unit:
      inp: (*1 *6 *3)
      out: (*2)
      weights: (1.0 0.5 0.5)
    unit:
      inp: (*1 *6)
      out: (*5)
      weights: (1.0 1.5)
    unit:
      inp: (*2 *5)
      out: (*4)
      weights: (0.5 -1.0)


In [57]:
print(dumps(values, crossref=1, pretty=2, hsize=3))

network:
  nodes:
    &1 node:
      label: "x1"
    &6 node:
      label: "x2"
    &3 node:
      label: "x3"
    &2 node:
      label: "u1"
    &5 node:
      label: "u2"
    &4 node:
      label: "y"
  units:
    unit:
      inp: (*1 *6 *3)
      out: (*2)
      weights: (1.0 0.5 0.5)
    unit:
      inp: (*1 *6)
      out: (*5)
      weights: (1.0 1.5)
    unit:
      inp: (*2 *5)
      out: (*4)
      weights: (0.5 -1.0)


### Mapping named complex values to types

In [60]:
from __future__ import print_function
from axon import dump_as_str, as_unicode, factory, reduce
import numpy as np

@factory('ndarray')
def create_array(mapping, sequence):
    shape = mapping.get('shape', None)
    dtype = mapping['dtype']
    if type(dtype) is list:
        dtype = [(str(n), str(t)) for n, t in dtype]
    a = np.array(sequence, dtype=dtype)
    if shape is not None:
        a.shape = shape
    return a

@reduce(np.ndarray)
def reduce_array(a):
    signes = {'<', '=', '>', '!'}
    if len(a.dtype.descr) > 1:
        dtype = [
            (as_unicode(n), (as_unicode(t[1:]) \
                             if t[0] in signes \
                             else as_unicode(t)))
            for n, t in a.dtype.descr]
        return element('ndarray', {'dtype':dtype}, a.tolist())
    else:
        dtype_str = a.dtype.str
        dtype_str = as_unicode(dtype_str[1:]) \
                if dtype_str[0] in signes \
                else as_unicode(dtype_str)
        return element('ndarray', {'shape': a.shape, 'dtype':dtype_str}, a.tolist())

dump_as_str(np.int8)
dump_as_str(np.int16)
dump_as_str(np.int32)
dump_as_str(np.int64)
dump_as_str(np.float16)
dump_as_str(np.float32)
dump_as_str(np.float64)
dump_as_str(np.float128)
dump_as_str(np.int_)
dump_as_str(np.float_)
dump_as_str(np.double)


In [62]:
a = np.array([[1, 2], [3, 4], [5, 6]])
print('\nCompact form:')
text = dumps([a])
print(text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

print('\nFormatted form:')
text = dumps([a], pretty=1, hsize=3)
print(text)
print('\nIndented form:')
text = dumps([a], pretty=2, hsize=3)
print(text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

a = np.array(
        [(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
        dtype=[('x', int), ('y', int), ('z', float)])
text = dumps([a])
print('val=', text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

text = dumps([a], pretty=1, hsize=3)
print('val=', text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))
text = dumps([b], pretty=2, hsize=3)
print('val=', text)



Compact form:
ndarray{dtype:"i8" shape:(3 2) [1 2] [3 4] [5 6]}
val= array([[1, 2],
       [3, 4],
       [5, 6]])

Formatted form:
ndarray:
  dtype: "i8"
  shape: (3 2)
  [1 2]
  [3 4]
  [5 6]

Indented form:
ndarray:
  dtype: "i8"
  shape: (3 2)
  [1 2]
  [3 4]
  [5 6]
val= array([[1, 2],
       [3, 4],
       [5, 6]])
val= ndarray{dtype:[("x" "i8") ("y" "i8") ("z" "f8")] (1 2 3.0) (3 4 5.0) (4 5 6.0)}
val= array([(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
      dtype=[('x', '<i8'), ('y', '<i8'), ('z', '<f8')])
val= ndarray:
  dtype: [
    ("x" "i8")
    ("y" "i8")
    ("z" "f8")]
  (1 2 3.0)
  (3 4 5.0)
  (4 5 6.0)
val= array([(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
      dtype=[('x', '<i8'), ('y', '<i8'), ('z', '<f8')])
val= ndarray:
  dtype: [
    ("x" "i8")
    ("y" "i8")
    ("z" "f8")]
  (1 2 3.0)
  (3 4 5.0)
  (4 5 6.0)


## Documents

### HTML document as data

In [63]:
html_as_data = '''
html {
  head {
    meta: {charset: "utf-8"}
    title: "Sample html page"
  }
  body {

  form {
    name:"test" method:"post" action:"result.html"
    p {
      b { "Your name:" } br{}
      input { type:"text" size:"40" }
    }
    p { 
      b { "What is your browser:" } 
      input { type:"radio" name:"browser" value:"ie" } "Internet Explorer" br{}
      input { type:"radio" name:"browser" value:"opera" } "Opera" br{}
      input { type:"radio" name:"browser" value:"firefox" } "Firefox" br{}
    }
    p {
      input { type:"submit" value:"Send" }
    }
  }
  }
}
'''
values = loads(html_as_data)[0]
pprint(values)

sequence('html', [mapping('head', {'meta': {'charset': 'utf-8'}, 'title': 'Sample html page'}), sequence('body', [element('form', {'method': 'post', 'name': 'test', 'action': 'result.html'}, [sequence('p', [sequence('b', ['Your name:']), empty('br'), mapping('input', {'size': '40', 'type': 'text'})]), sequence('p', [sequence('b', ['What is your browser:']), mapping('input', {'value': 'ie', 'name': 'browser', 'type': 'radio'}), 'Internet Explorer', empty('br'), mapping('input', {'value': 'opera', 'name': 'browser', 'type': 'radio'}), 'Opera', empty('br'), mapping('input', {'value': 'firefox', 'name': 'browser', 'type': 'radio'}), 'Firefox', empty('br')]), sequence('p', [mapping('input', {'value': 'Send', 'type': 'submit'})])])])])


In [64]:
print(dumps(values))

head{meta:{charset:"utf-8"} title:"Sample html page"}
body{form{action:"result.html" method:"post" name:"test" p{b{"Your name:"} br{} input{size:"40" type:"text"}} p{b{"What is your browser:"} input{name:"browser" type:"radio" value:"ie"} "Internet Explorer" br{} input{name:"browser" type:"radio" value:"opera"} "Opera" br{} input{name:"browser" type:"radio" value:"firefox"} "Firefox" br{}} p{input{type:"submit" value:"Send"}}}}


In [65]:
print(dumps(values, pretty=1, hsize=4))

head:
  meta: {charset: "utf-8"}
  title: "Sample html page"
body:
  form:
    action: "result.html"
    method: "post"
    name: "test"
    p:
      b:
        "Your name:"
      br:
      input:
        size: "40"
        type: "text"
    p:
      b:
        "What is your browser:"
      input:
        name: "browser"
        type: "radio"
        value: "ie"
      "Internet Explorer"
      br:
      input:
        name: "browser"
        type: "radio"
        value: "opera"
      "Opera"
      br:
      input:
        name: "browser"
        type: "radio"
        value: "firefox"
      "Firefox"
      br:
    p:
      input:
        type: "submit"
        value: "Send"


In [66]:
print(dumps(values, pretty=2, hsize=4))

head:
  meta: {charset: "utf-8"}
  title: "Sample html page"
body:
  form:
    action: "result.html"
    method: "post"
    name: "test"
    p:
      b:
        "Your name:"
      br:
      input:
        size: "40"
        type: "text"
    p:
      b:
        "What is your browser:"
      input:
        name: "browser"
        type: "radio"
        value: "ie"
      "Internet Explorer"
      br:
      input:
        name: "browser"
        type: "radio"
        value: "opera"
      "Opera"
      br:
      input:
        name: "browser"
        type: "radio"
        value: "firefox"
      "Firefox"
      br:
    p:
      input:
        type: "submit"
        value: "Send"


### SVG example

In [67]:
text = '''
svg:
  xmlns:"http://www.w3.org/2000/svg" 
  viewBox:(0 0 300 150)
  defs:
    radialGradient:
        id: "gradient"
        cx: "50%" cy: "50%"
        stop:
          offset:"0%" 
          style:
            stop_color: rgb{200 200 200} 
            stop_opacity:0
        stop:
          offset:"100%"
          style:
            stop_color: rgb{0 0 255}
            stop_opacity:1
  ellipse:
    cx:100 cy:50 rx:100 ry:50
    style:
        fill: url{"#gradient"}
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
        translate{90 30}
        rotate{-30 10}
        scale{0.9}]
  text:
    x:0 y:30 font_size:"18px" font_weight:"bold"
    text_anchor:"middle"
    "Button"
'''
values = loads(text)
pprint(values)


[element('svg', {'viewBox': (0, 0, 300, 150), 'xmlns': 'http://www.w3.org/2000/svg'}, [sequence('defs', [element('radialGradient', {'id': 'gradient', 'cy': '50%', 'cx': '50%'}, [element('stop', {'offset': '0%'}, [mapping('style', {'stop_color': sequence('rgb', [200, 200, 200]), 'stop_opacity': 0})]), element('stop', {'offset': '100%'}, [mapping('style', {'stop_color': sequence('rgb', [0, 0, 255]), 'stop_opacity': 1})])])]), element('ellipse', {'cy': 50, 'ry': 50, 'rx': 100, 'cx': 100}, [mapping('style', {'fill': sequence('url', ['#gradient'])})]), mapping('path', {'d': 'M150 0 L75 200 L225 200 Z'}), mapping('g', {'transform': [sequence('translate', [90, 30]), sequence('rotate', [-30, 10]), sequence('scale', [0.9])]}), element('text', {'x': 0, 'text_anchor': 'middle', 'font_weight': 'bold', 'font_size': '18px', 'y': 30}, ['Button'])])]


In [68]:
print(dumps(values))

svg{viewBox:(0 0 300 150) xmlns:"http://www.w3.org/2000/svg" defs{radialGradient{cx:"50%" cy:"50%" id:"gradient" stop{offset:"0%" style{stop_color:rgb{200 200 200} stop_opacity:0}} stop{offset:"100%" style{stop_color:rgb{0 0 255} stop_opacity:1}}}} ellipse{cx:100 cy:50 rx:100 ry:50 style{fill:url{"#gradient"}}} path{d:"M150 0 L75 200 L225 200 Z"} g{transform:[translate{90 30} rotate{-30 10} scale{0.9}]} text{font_size:"18px" font_weight:"bold" text_anchor:"middle" x:0 y:30 "Button"}}


In [69]:
print(dumps(values, pretty=1, hsize=2))

svg:
  viewBox: (
    0 0
    300 150)
  xmlns: "http://www.w3.org/2000/svg"
  defs:
    radialGradient:
      cx: "50%"
      cy: "50%"
      id: "gradient"
      stop:
        offset: "0%"
        style:
          stop_color: rgb:
            200 200
            200
          stop_opacity: 0
      stop:
        offset: "100%"
        style:
          stop_color: rgb:
            0 0
            255
          stop_opacity: 1
  ellipse:
    cx: 100
    cy: 50
    rx: 100
    ry: 50
    style:
      fill: url:
        "#gradient"
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
      translate:
        90 30
      rotate:
        -30 10
      scale:
        0.9]
  text:
    font_size: "18px"
    font_weight: "bold"
    text_anchor: "middle"
    x: 0
    y: 30
    "Button"


In [70]:
print(dumps(values, pretty=2, hsize=2))

svg:
  viewBox: (
    0 0
    300 150)
  xmlns: "http://www.w3.org/2000/svg"
  defs:
    radialGradient:
      cx: "50%"
      cy: "50%"
      id: "gradient"
      stop:
        offset: "0%"
        style:
          stop_color: rgb:
            200 200
            200
          stop_opacity: 0
      stop:
        offset: "100%"
        style:
          stop_color: rgb:
            0 0
            255
          stop_opacity: 1
  ellipse:
    cx: 100
    cy: 50
    rx: 100
    ry: 50
    style:
      fill: url:
        "#gradient"
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
      translate:
        90 30
      rotate:
        -30 10
      scale:
        0.9]
  text:
    font_size: "18px"
    font_weight: "bold"
    text_anchor: "middle"
    x: 0
    y: 30
    "Button"


## Element tree

In [71]:
from xml.etree import ElementTree, cElementTree

Let's define function for dumping ``ElementTree`` structures according to ``AXON`` rules.

In [84]:
@reduce(cElementTree.Element)
@reduce(ElementTree.Element)
def element_reduce(elem):
    children = elem.getchildren()
    if elem.attrib:
        if children:
            return element(elem.tag, elem.attrib, children)
        else:
            return mapping(elem.tag, elem.attrib)
    elif children:
            return sequence(elem.tag, children)
    else:
        return elem.tag
        
@reduce(cElementTree.ElementTree)
@reduce(ElementTree.ElementTree)
def etree_reduce(element):
    return element_reduce(element.getroot())
    
    

In [85]:
root = ElementTree.Element('persons')
ElementTree.SubElement(root, 'person', {'name':'Alice', 'age':25})
ElementTree.SubElement(root,'person', {'name':'Bob', 'age':33})
etree = ElementTree.ElementTree(root)

text = dumps([etree], pretty=1)
print(text)

persons:
  person:
    age: 25
    name: "Alice"
  person:
    age: 33
    name: "Bob"


In [86]:
def update_attribs(d):
    for key, val in d.items():
        d[key] = str(val)

class ElementTreeBuilder(GenericBuilder):
    def empty(self, name):
        return cElementTree.Element(name)
    def mapping(self, name, attribs):
        update_attribs(attribs)
        return cElementTree.Element(name, attribs)
    def sequence(self, name, children):
        e = cElementTree.Element(name)
        e.extend(children)
        return e
    def element(self, name, attribs, children):
        update_attribs(attribs)
        e = cElementTree.Element(name, attribs)
        e.extend(children)
        return e

register_builder('etree', ElementTreeBuilder())    

In [87]:
vals = loads(text, mode='etree')

In [88]:
cElementTree.dump(vals[0])

<persons><person age="25" name="Alice" /><person age="33" name="Bob" /></persons>
